<a href="https://colab.research.google.com/github/takatakamanbou/Data/blob/2021/ex06notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data2021 ex06notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/Data-logo06.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?Data/2021

## 準備

Google Colab の Notebook では， Python というプログラミング言語のコードを動かして計算したりグラフを描いたりもできます．
Python は，データ分析や機械学習・人工知能分野ではメジャーなプログラミング言語ですが，それを学ぶことはこの授業の守備範囲ではありません．ですので，以下の所々に現れるプログラムっぽい記述の内容は，理解できなくて構いません．

以下，コードセルを上から順に実行していってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

In [ ]:
# データを読み込む
dfHW = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/ex06hw.csv', header=0)
dfHW.drop(columns=['num'], inplace=True)
dfHW

----
## データの正規化
----




----
### データを正規化したくなる気持ち



In [ ]:
dfHW

上記は，1000人分の $(\mbox{身長}, \mbox{体重})$ のデータです（注）．
なんか身長の値が大きくて，体重の値が小さいなあ，と思ってよく見ると，身長の単位はミリメートル，体重の単位はトンで表されています.


注) このデータは，人工的に作ったものです．平均や標準偏差は17歳男子の統計データに近くなるようにしてありますが，分布は実際のデータとは違ってます．

何も考えず身長，体重それぞれのヒストグラムを描いてみると，こんなんなります．

In [ ]:
fig, ax = plt.subplots(2, 1, facecolor="white", figsize=(8, 6))
ax[0].hist(dfHW['身長[mm]'], label='height')
ax[0].legend()
ax[0].set_xlim(1300, 2100)
ax[0].set_ylim(0, 300)
ax[1].hist(dfHW['体重[ton]'], label='weight')
ax[1].legend()
ax[1].set_xlim(0.02, 0.10)
ax[1].set_ylim(0, 300)
plt.show()

当たり前ですが，値の大きさとか，範囲とか，桁違いですね．
当然，平均値や標準偏差の値も...

In [ ]:
mH, mW = np.mean(dfHW['身長[mm]']), np.mean(dfHW['体重[ton]'])
sH, sW = np.std(dfHW['身長[mm]']), np.std(dfHW['体重[ton]'])
print(f'身長の平均 = {mH:.1f}, 身長の標準偏差 = {sH:.3f}')
print(f'体重の平均 = {mW:.6f}, 体重の標準偏差 = {sW:.6f}')

値が大きすぎたり小さすぎたりすると扱いにくいし解釈もしづらいので，値が適当な範囲になるように一定数を足したり，定数倍したりすることを考えましょう．

とりあえずすぐに思いつくのは，身長の単位を[cm]にして，体重の単位を[kg]にすることです．
身長を $\frac{1}{10}$ に，体重を $1000$ 倍にしてみましょう．

In [ ]:
X = dfHW['身長[mm]']/10   # [cm]単位にした身長
Y = dfHW['体重[ton]']*1000 # [kg]単位にした体重

In [ ]:
fig, ax = plt.subplots(2, 1, facecolor="white", figsize=(8, 6))
ax[0].hist(X, label='height')
ax[0].legend()
ax[0].set_xlim(130, 210)
ax[0].set_ylim(0, 300)
ax[1].hist(Y, label='weight')
ax[1].legend()
ax[1].set_xlim(20, 100)
ax[1].set_ylim(0, 300)
plt.show()

In [ ]:
mH, mW = np.mean(X), np.mean(Y)
sH, sW = np.std(X), np.std(Y)
print(f'身長の平均 = {mH:.2f}, 身長の標準偏差 = {sH:.2f}')
print(f'体重の平均 = {mW:.2f}, 体重の標準偏差 = {sW:.2f}')

見慣れた値になりました．
[第4回](https://www-tlab.math.ryukoku.ac.jp/wiki/?Data/2021#ex04) の ex04notebookA.ipynb で証明した性質：

> データを定数 $a$ 倍すると，平均は元の $a$ 倍になり，標準偏差は元の $|a|$ 倍になる

がちゃんと成り立ってますね．


----
### 正規化

身長と体重のようなデータは，たまたま[cm]と[kg]を単位とすれば，それなりの範囲におさまりました．では，様々なデータが与えられたときに，値の範囲をある程度そろえたいと思ったら，どうするのがよいでしょう？

問題や目的に応じて，様々なやり方がありますが，最も一般的な方法は，**データの平均が $0$ に，標準偏差が $1$ になるようにする**，というものです（標準偏差が $1$ $\Leftrightarrow$ 分散が $1^2=1$ なので，「分散が $1$ になるようにする」と言うこともあります）．

このようにデータの値をいじることを，データを **正規化する** といいます．

データ $x_1, x_2, \dots , x_N$ の平均値が $\bar{x}$ で，標準偏差が $s$ だったとすると，
$$
z_n = \frac{(x_n - \bar{x})}{s}\qquad (n = 1, 2, \dots , N)
$$
とすれば，$z_1, z_2, \dots , z_N$ の平均は $0$ で，標準偏差は $1$ になります．

［証明］

$z_n$の平均は，

$$
\begin{aligned}
\frac{1}{N}\sum_{n=1}^{N}z_n &= \frac{1}{N}\sum_{n=1}^{N}\frac{(x_n - \bar{x})}{s} = \frac{1}{sN}\sum_{n=1}^{N}(x_n - \bar{x}) \\
&= \frac{1}{s}\left( \frac{1}{N}\sum_{n=1}^{N}x_n - \frac{1}{N}\sum_{n=1}^{N}\bar{x} \right)
= \frac{1}{s}\left(\bar{x} - \frac{N\bar{x}}{N}\right)\\
&= \frac{1}{s}\times 0 = 0
\end{aligned}
$$

である．分散は，

$$
\begin{aligned}
\frac{1}{N}\sum_{n=1}^{N}(z_n - 0)^2 &= \frac{1}{N}\sum_{n=1}^{N}\frac{(x_n - \bar{x})^2}{s^2} = \frac{1}{s^2}\frac{1}{N}\sum_{n=1}^{N}(x_n-\bar{x})^2 \\
&= \frac{1}{s^2} \times s^2 = 1
\end{aligned}
$$

である．よって，標準偏差も $1$ である．




実際に，上記の身長，体重をそれぞれ正規化してみましょう．

In [ ]:
# 身長[cm]の平均と標準偏差
mH = np.mean(X)
sH = np.std(X)
print(f'身長の平均 = {mH:.2f}, 身長の標準偏差 = {sH:.2f}')

# 身長の正規化
X_normalized = (X - mH) / sH
print(f'正規化した身長の平均 = {np.mean(X_normalized):.2f}, 正規化した身長の標準偏差 = {np.std(X_normalized):.2f}')

In [ ]:
# 体重[kg]の平均と標準偏差
mW = np.mean(Y)
sW = np.std(Y)
print(f'体重の平均 = {mW:.2f}, 体重の標準偏差 = {sW:.2f}')

# 体重の正規化
Y_normalized = (Y - mW) / sW
print(f'正規化した体重の平均 = {np.mean(Y_normalized):.2f}, 正規化した体重の標準偏差 = {np.std(Y_normalized):.2f}')

正規化した身長体重のヒストグラムはこうなります．横軸の範囲に注目．

In [ ]:
fig, ax = plt.subplots(2, 1, facecolor="white", figsize=(8, 6))
ax[0].hist(X_normalized, label='height')
ax[0].legend()
ax[0].set_xlim(-4, 4)
ax[0].set_ylim(0, 300)
ax[1].hist(Y_normalized, label='weight')
ax[1].legend()
ax[1].set_xlim(-4, 4)
ax[1].set_ylim(0, 300)
plt.show()

この例では，正規化すると，身長も体重もよく似た分布の形をしていることがわかります．
データを正規化すると，単位の違いなどにとらわれずに分布の本質的な性質を調べやすくなります．

----
### 正規化と相関係数

正規化したら相関係数はどうなるか，計算してみましょう．

まずは，[mm]/[ton]単位の身長/体重の散布図と相関係数．
横軸縦軸の範囲に注意．

In [ ]:
fig, ax = plt.subplots(facecolor="white", figsize=(6, 6))
ax.scatter(dfHW['身長[mm]'], dfHW['体重[ton]'])
#ax.set_xlim(140, 200)
#ax.set_ylim(20, 100)
ax.set_xlim(1300, 2100)
ax.set_ylim(0.02, 0.1)
plt.show()

r = np.corrcoef(dfHW['身長[mm]'], dfHW['体重[ton]'])[1, 0]
print(f'相関係数は {r:.3f}')

次は，正規化後の散布図と相関係数．

In [ ]:
fig, ax = plt.subplots(facecolor="white", figsize=(6, 6))
ax.scatter(X_normalized, Y_normalized)
ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
plt.show()

r = np.corrcoef(X_normalized, Y_normalized)[1, 0]
print(f'相関係数は {r:.3f}')

★★★ 上記の相関係数の値を，ノート等にメモしておきましょう． Quiz で使う...かもしれません ★★★

分布の傾きが変わっていますが，これは単に縦軸横軸の範囲指定による見え方の問題です．
相関係数を見ると，正規化の前後で全く値が変わっていないのがわかります．

このように，相関係数の値は，データの値を定数倍したり一定値を加えたりしても，変化しません．

そのいい加減な説明：

相関係数の式は，
$$
r = \frac{s_{XY}}{s_X s_Y}
$$
ただし
$$
\begin{aligned}
s_X &= \sqrt{\frac{1}{N}\sum_{n=1}^{N}(x_n - \bar{x})^2}\\
s_Y &= \sqrt{\frac{1}{N}\sum_{n=1}^{N}(y_n - \bar{y})^2}\\
s_{XY} &= \frac{1}{N}\sum_{n=1}^{N}(x_n - \bar{x})(ｙ_n - \bar{y})
\end{aligned}
$$
というものでした．
分母の共分散も分子の標準偏差も平均を引いた値であるため，平均を変化させてもこれらの値は変化しません．
また，定数倍すると分母分子が等しく定数倍される結果，やはり値は変化しません．